In [ ]:
import numpy as np
import random
import copy
import cv2
from collections import deque

class AntColonySegmentation:
    def __init__(self, image, num_ants, max_iterations, alpha, beta, rho):
        self.image = image
        self.num_ants = num_ants
        self.max_iterations = max_iterations
        self.alpha = alpha  # pheromone importance
        self.beta = beta  # visibility (intensity) importance
        self.rho = rho  # pheromone evaporation rate

        self.pheromones = np.ones([self.image.shape[0],self.image.shape[1]])
        

    def initialize_ants(self):
        ants = []
        for _ in range(self.num_ants):
            ant = {'path': [], 'intensity_sum': 0.0}
            ants.append(ant)
        return ants

    
    def clear_ant_paths(self, ants):
        for ant in ants:
            ant['path'] = []
            ant['score'] = 0.0


    def construct_segment(self, ant):
        current_position = (np.random.randint(0, self.image.shape[0]), np.random.randint(0, self.image.shape[1]))
            
        ant['path'].append(current_position)
        
        max_backtrack = 10
        
        while True:
            neighbors = self.get_neighbors(current_position)
            probabilities = self.calculate_probabilities(current_position,neighbors,ant)


            if sum(probabilities) == 0:
                backtracked = False
                for i in range(2, min(max_backtrack, len(ant['path']))):
                    current_position = ant['path'][-i]
                    neighbors = self.get_neighbors(current_position)
                    probabilities = self.calculate_probabilities(current_position, neighbors, ant)
                    if sum(probabilities) > 0:
                        backtracked = True
                        break
                if not backtracked:
                    break

            # if sum(probabilities) == 0:
            #     break
            next_position = neighbors[np.random.choice(len(neighbors), p=probabilities)]

            
            if next_position not in ant['path']:
                ant['path'].append(next_position)
                current_position = next_position
        
                

    def get_neighbors(self, position):
        height, width,_ = self.image.shape
        row, col = position
        neighbors = []
        #neighbors_offsets = [(0, 1), (0, -1),(1,0),(-1,0)]
        neighbors_offsets = [(0, 1), (0, -1), (1, 0), (-1, 0),(1,1),(1,-1),(-1,-1),(-1,1)]

        for offset_row, offset_col in neighbors_offsets:
            new_row, new_col = row + offset_row, col + offset_col
            if 0 <= new_row < self.image.shape[0] and 0 <= new_col < self.image.shape[1]:
                neighbors.append((new_row, new_col))

        return neighbors

    def calculate_probabilities(self, current_position,neighbors,ant):
        intensity = self.image[current_position][1]
        probabilities = []

        
        for neighbor in neighbors:
            attractiveness = self.calc_attractiveness(neighbor)
            
            pheromone = self.pheromones[neighbor]
            probability = (pheromone ** self.alpha) * (attractiveness ** self.beta)
            
            if neighbor in ant['path']:
                probability = 0
                
            probabilities.append(probability)
            
            

        total = sum(probabilities)
        if total > 0:
            probabilities = [p/total for p in probabilities]
        else:
            probabilities = [0 for _ in probabilities]
            
        
        return probabilities

    def calc_attractiveness(self,position):
        row, col = position
        pixel = self.image[row, col].astype(np.float32)
        B,G,R = pixel[0], pixel[1], pixel[2]


        water_index = (R + G + 1) / (B + 1)
        
        return min(max(water_index / 2.0, 0.1), 1.0)
        

    def is_water_pixel(self, position):
        row, col = position
        pixel = self.image[row, col].astype(np.float32)
        B,G,R = pixel[0], pixel[1], pixel[2]

        
        return (R > 80) and (G > 80) and (R >= G)  and (R >= B) and (G >= B) and (B>40) #and (np.abs(R - G) < 50)
        

    def update_pheromones(self, ants):
        self.pheromones *= (1 - self.rho)
        for ant in ants:
            for position in ant['path']:
                self.pheromones[position] += ant['score']  #sklonio 1.0/

    def region_grow(self,image, seed_mask, is_water_pixel, max_neighbors=8):
        
        # Region growing segmentation.
        # - max_neighbors: 4 or 8 connectivity
        
        h, w = image.shape[:2]
        visited = np.zeros((h, w), dtype=np.uint8)
        result_mask = np.zeros((h, w), dtype=np.uint8)
    
        queue = deque([(r, c) for r in range(h) for c in range(w) if seed_mask[r, c] > 0])
    
        
        if max_neighbors == 4:
            neighbors = [(0,1),(0,-1),(1,0),(-1,0)]
        else:
            neighbors = [(0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]
    
        while queue:
            r, c = queue.popleft()
            if visited[r, c]:
                continue
            visited[r, c] = 1
    
            if is_water_pixel((r,c)):
                result_mask[r, c] = 255
                for dr, dc in neighbors:
                    rr, cc = r+dr, c+dc
                    if 0 <= rr < h and 0 <= cc < w and not visited[rr, cc]:
                        queue.append((rr, cc))
    
        return result_mask

    def run(self):
        ants = self.initialize_ants()
        segmentation_mask = np.zeros(self.image.shape[:2], dtype=np.uint8)
        
        
        for iteration in range(self.max_iterations):
            
            self.clear_ant_paths(ants)
            
            for ant in ants:
                self.construct_segment(ant)
                ant['score'] = sum(self.is_water_pixel(pos) for pos in ant['path'])
                
            self.update_pheromones(ants)
            

            # Choose the best segment based on its score
            current_best = max(ants, key=lambda ant: ant['score'])
            self.best = current_best['path']
            for pos in current_best['path']:
                segmentation_mask[pos] = 255


            #print(f"Iteration {iteration+1}/{self.max_iterations},current best path length: {len(current_best['path'])}")

        segmentation_mask = self.region_grow(self.image,segmentation_mask,self.is_water_pixel)    

        return segmentation_mask


In [ ]:
num_ants = 25
max_iterations = 10

alpha = 2
beta = 2
rho = 0.1

for i in range(10):
    path = f'dataset/images/{i}.jpg'
    image = cv2.imread(path)
    aco_segmentation = AntColonySegmentation(image, num_ants, max_iterations, alpha, beta, rho)
    result = aco_segmentation.run()
    cv2.imwrite(f'aco_results/{i}.png',result)
print('finished')

# image = cv2.imread("dataset/images/9.jpg")
# aco_segmentation = AntColonySegmentation(image, num_ants, max_iterations, alpha, beta, rho)
# result = aco_segmentation.run()
#cv2.imwrite('aco_results/9.png',result)

# cv2.imshow("Result", result)
# cv2.imshow("Original", image)

# cv2.waitKey(0)
# cv2.destroyAllWindows()